In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob

In [2]:
# Define a function to read data
def load_data(base_directory):
    data = []
    labels = []
    categories = ['business', 'tech', 'sport', 'politics', 'entertainment']
    for category in categories:
        directory = os.path.join(base_directory, category)
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            if filepath.endswith(".txt"):
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    text = file.read()
                    data.append(text)
                    labels.append(category)
    return data, labels

# Add a function to calculate document length and sentiment score
def feature_engineering(data):
    lengths = [len(text.split()) for text in data]  # Document length
    sentiments = [TextBlob(text).sentiment.polarity for text in data]  # Sentiment score
    return np.array(lengths), np.array(sentiments)

# Load data
current_directory = '.'  # Current working directory
data, labels = load_data(current_directory)

# Feature extraction
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_features = vectorizer.fit_transform(data)
lengths, sentiments = feature_engineering(data)

# Merge all features into one large feature matrix
features = np.hstack((tfidf_features.toarray(), lengths[:, np.newaxis], sentiments[:, np.newaxis]))

# Normalize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Encode labels
y = pd.factorize(labels)[0]

# Split testset
X_train, X_test, y_train, y_test = train_test_split(features_scaled, y, test_size=0.2, random_state=42)


C:\Users\PENG\AppData\Local\Temp\ipykernel_29184\993665039.py:40: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  y = pd.factorize(labels)[0]


In [3]:
# Train using a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Print classification report
print(classification_report(y_test, predictions, target_names=pd.unique(labels)))

               precision    recall  f1-score   support

     business       0.97      0.97      0.97       115
         tech       0.96      0.93      0.95        74
        sport       0.94      1.00      0.97        99
     politics       0.95      0.94      0.94        82
entertainment       0.96      0.92      0.94        75

     accuracy                           0.96       445
    macro avg       0.96      0.95      0.95       445
 weighted avg       0.96      0.96      0.96       445



C:\Users\PENG\AppData\Local\Temp\ipykernel_29184\3425051959.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  print(classification_report(y_test, predictions, target_names=pd.unique(labels)))
